# Tetris

We apply the cross entropy method to solve the game of tetris.

This follows the work of 

> Szita, István, and András Lörincz. "Learning Tetris using the noisy cross-entropy method." Neural computation 18.12 (2006): 2936-2941.

The github repository for Tetris that we use is:

https://github.com/jaybutera/tetrisRL

(Please follow the install instructions there.) 

To play tetris yourself, type the following in the command line:
```
$ python3 user_engine.py
```



In [1]:
from copy import deepcopy
from time import time

from tetrisRL.engine import *

In [2]:
def screen_print(obs):
    for y in range(len(obs[0])):
        for x in range(len(obs)):
            if obs[x,y] == 0:
                print(chr(9617),end='')
            else :
                print(chr(9608),end='')
        print('')
        
    print('')
    print('shape: ',print(reverse_shapes(str(env.shape))) )
    

In [3]:
shapes = {
    'T': [(0, 0), (-1, 0), (1, 0), (0, -1)],
    'J': [(0, 0), (-1, 0), (0, -1), (0, -2)],
    'L': [(0, 0), (1, 0), (0, -1), (0, -2)],
    'Z': [(0, 0), (-1, 0), (0, -1), (1, -1)],
    'S': [(0, 0), (-1, -1), (0, -1), (1, 0)],
    'I': [(0, 0), (0, -1), (0, -2), (0, -3)],
    'O': [(0, 0), (0, -1), (-1, 0), (-1, -1)],
}

shape_names = ['T', 'J', 'L', 'Z', 'S', 'I', 'O']
# This can be done more clean..
rs_0 = {str(val) : key for key, val in shapes.items()}
rs_1 = {str(rotated(val)) : key for key, val in shapes.items()}
rs_2 = {str(rotated(val,cclk=True)) : key for key, val in shapes.items()}
rs_3 = {str(rotated(rotated(val))) : key for key, val in shapes.items()}


In [9]:
shape_letters = {**rs_0, **rs_1, **rs_2, **rs_3}

In [94]:
# Action representation a rotation and a column
rotations = [-1,0,1,2]
columns = [0,1,2,3,4,5,6,7,8,9]

In [1225]:

class mover:
    '''sequence of moves to correct rotation and column'''
    def __init__(self,env,rotation,column):
        self.rotation = rotation
        self.column = column
        self.env = env
        self.done_rotating = False
        self.done_translating = False
        self.old_anchor = (None,None)
        self.action = None
        self.time_step=0
        
    def act(self):        
        # if done rotating move to correct colummn
        if self.done_rotating:
            self._translate(self.column)
            print('a')
              
        # rotate into the correct position
        if not self.done_rotating:
            self._rotate(self.rotation)
            print('b')    
    
        # if done translating reset
        if self.done_translating :
            action = self.action
            self._reset()
            return action, True
        else:
            return self.action, False
    
    def _rotate(self,rotation):
        if rotation == -1 :
            self.action = 4
            self.done_rotating = True
        elif rotation == 1 :
            self.action = 5 
            self.done_rotating = True
        elif rotation == 2 :
            self.time_step+=1
            self.action = 5
            if self.time_step == 2:
                self.done_rotating = True 
                self.time_step =0
        elif rotation == 0 :
            self.done_rotating = True
            self.action = 6
    
    def _translate(self,column):
        # if correct or not moving or anchor out of bounds then drop 
        if self.env.anchor[0]==column \
            or self.env.anchor[0]==self.old_anchor[0] \
            or (not 0<=self.env.anchor[0]<=self.env.width):
            self.action = 2
            self.done_translating = True

        # keep moving if not done
        if not self.done_translating:
            if self.env.anchor[0] > column:
                self.action = 0
            elif self.env.anchor[0] < column :
                self.action = 1
            
        self.old_anchor = self.env.anchor
        
    def _reset(self):
        self.done_rotating = False
        self.done_translating = False
        self.old_anchor = (None,None)
        self.action = None
        self.time_step=0
        print('resetting')

    

In [1274]:
env.clear()
self = mover(env,rotation=1,column=2)

In [1459]:

action, done_moving = self.act()
env.step(action)
print(self.done_rotating,self.done_translating)
print(env.anchor,self.old_anchor)
print(action)
print(self.time_step)
print(env)

b
True False
(5, 1) (None, None)
5
0
o----------o
|          |
|     XXX  |
|     X    |
|          |
|          |
|          |
|          |
|          |
|          |
|          |
|          |
|          |
|  X       |
|  XX      |
|   X      |
|  XX      |
|  XX      |
|  X       |
|  X       |
|  XX      |
o----------o


In [1199]:
self.env.anchor[0]

5

In [1189]:
self.old_anchor[0]

In [957]:
self.done

AttributeError: 'mover' object has no attribute 'done'

In [258]:
action, done_moving = self.act()
env.step(action)
print(action)
print(env.anchor)

0
(3, 2)


In [184]:
env.step(action)
env

KeyError: None

In [87]:
width, height = 10, 20
env = TetrisEngine(width, height)

In [42]:
# Reset the environment
obs = env.clear()

while True:
    # Get an action from a theoretical AI agent
    action = np.random.randint(7)

    # Sim step takes action and returns results
    obs, reward, done = env.step(action)

    # Done when game is lost
    if done:
        break

In [43]:
width, height = 10, 20
env = TetrisEngine(width, height)

In [44]:
action = np.random.randint(7)

In [50]:
env.shape

[(0, 0), (-1, 0), (0, -1), (1, -1)]

In [46]:
env.anchor

(5.0, 0)

In [61]:
mv = mover(env,0,3)

mv.act()

In [58]:
env.anchor

(5.0, 0)

In [39]:
env.anchor

(5.0, 0)

In [105]:
tic = time()
for _ in range(100):
    deepcopy(env)
toc = time()
print(toc-tic)

0.016115188598632812


In [106]:
t_cop = deepcopy(env)

In [113]:
screen_print(t_cop.board)

░░░░░░░░░░
░░░░░░░░░░
░░░░░░░░░░
░░░░░░░░░░
░░░░░░░░░░
░░░░░░░░░░
░░░░░░░░░░
░░░░░░░░░░
░░░░░░░░░░
░░░░░░░░░░
░░░░░░░░░░
░░░░░░░░░░
░░░░░░░░░░
░░░░░░░░░░
░░░░░░░░░░
░░░░░░░░░░
░░░░░░░░░░
░░░░░░░░░░
░░░░██░░░░
░░░░░██░░░


In [90]:
for i in range(9600,9620):
    print(chr(i))

▀
▁
▂
▃
▄
▅
▆
▇
█
▉
▊
▋
▌
▍
▎
▏
▐
░
▒
▓


In [92]:
chr(9617)

'░'

In [114]:
t_cop

o----------o
|    XXX   |
|          |
|          |
|          |
|          |
|          |
|          |
|          |
|          |
|          |
|          |
|          |
|          |
|          |
|          |
|          |
|          |
|          |
|    XX    |
|     XX   |
o----------o

In [115]:
t_cop.shape

[(0, 0), (-1, 0), (1, 0), (0, -1)]

In [149]:
reverse_shapes[str(t_cop.shape)]

'T'